## Staph data
http://users-birc.au.dk/cstorm/courses/MLiB_f14/project1.html

In [6]:
import requests
import numpy as np
import pandas as pd

In [4]:
staph_url = ["http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome1.fa",
            "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome2.fa",
           "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome3.fa"]
staph_annot = ["http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation1.fa",
              "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation2.fa"
              "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation3.fa"]

In [8]:
def load_file(url,genome=True):
    """input: url for genome, 
    output: pulled data without header and as a single txt string"""
    data = ''.join(requests.get(url).text.split("\n")[1::])
    return data

def segment_genome(data_string,split_size):
    """input: genome data string (data_string) and then size we should split the data (split_size)
    output: array split by specified split_size"""
    chunks = len(data_string)
    data = [data_string[i:i+split_size] for i in range(0,chunks,split_size)]
    return data

def gene_label(annotation, min_gene=10):
    """Input: annotation
    ouput: the label array for data (true if C or R else false for > 50% of line)"""
    size = len(annotation[0])
    print(size)
    label = [True if len(line.strip("C")) < size/2 or len(line.strip("R")) < size/2 else False 
             for line in annotation]
    return label

def gene_label_dir(annotation,direction="C"):
    """Input: annotation and direction is C (forward) unless specified as R (reverse)
    ouput: the label array for data"""
    size = len(annotation[0])
    label = [True if len(line.strip(direction)) < size/2 else False 
             for line in annotation]
    return label

def nucleotide_frequency(seq):
    '''Count the occurrences of characters in "seq".'''
    counts = {'A':0,'C':0,'G':0,'T':0}
    for c in seq:
        counts[c] +=1
    return counts

In [9]:
### Pipeline
data = load_file(staph_annot[0])
seg_50_an = segment_genome(data,50)

data = load_file(staph_url[0])
seg_50=segment_genome(data,50)

result = [[l[1] for l in list(nucleotide_frequency(seg).items())] for seg in seg_50]
df = pd.DataFrame(result)
df.columns=["a","c","g","t"]
df["gene"] = gene_label(seg_50_an)

gene_df = pd.DataFrame({"label":df.gene,"sequence": seg_50})

50


In [ ]:
df["sequence"] = gene_df.sequence
df["start"] = gene_df.sequence.apply(lambda seq: "ATG" in seq)
df["stop"] = gene_df.sequence.apply(lambda seq: any(x in seq for x in ["TAA", "TAG", "TGA"]))
# df.iloc[0].sequence

In [50]:
# from collections import defaultdict
def aa_find(seq):
    codon_count = {}
    for aa, codons in codon_map.items():
        codon_count[aa] = 0
        if any(x in seq for x in codons):
            codon_count[aa] += 1
    return codon_count

In [57]:
df["dict"] = gene_df.sequence.apply(lambda seq: aa_find(seq)) 

In [62]:
aa_counts = pd.DataFrame(list(df["dict"]))

In [84]:
# aa_counts[["a"],["c"],["g"],["t"],["gene"]] = [[df.a],[df.c],[df.g],[df.t],[df.gene]]
# aa_counts[["a"]]
aa_counts["a"] = df.a
aa_counts["c"] = df.c
aa_counts["g"] = df.g
aa_counts["t"] = df.t
aa_counts["gene"] = df.gene


In [78]:
df.shape

(37049, 9)

In [49]:
a = []
a.append(aa_find(df.iloc[1].sequence).values())
a.append(aa_find(df.iloc[2].sequence).values())
a.append(aa_find(df.iloc[3].sequence).values())
pd.DataFrame(a)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
1,0,1,1,0,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2,0,1,1,0,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1


In [41]:
[a[1] for a in aa_find(df.iloc[1].sequence).items()]

[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]

In [42]:
aa_find(df.iloc[1].sequence).items()

[('A', 1),
 ('C', 1),
 ('E', 1),
 ('D', 0),
 ('G', 1),
 ('F', 1),
 ('I', 1),
 ('H', 1),
 ('K', 1),
 ('M', 1),
 ('L', 1),
 ('N', 1),
 ('Q', 1),
 ('P', 1),
 ('S', 1),
 ('R', 1),
 ('T', 1),
 ('W', 1),
 ('V', 1),
 ('Y', 0),
 ('Stop', 1)]

In [26]:
codon_to_aa.items()
codon_map.items()

[('A', ['GCT', 'GCC', 'GCA', 'GCG']),
 ('C', ['TGT', 'TGC']),
 ('E', ['GAA', 'GAG']),
 ('D', ['GAT', 'GAC']),
 ('G', ['GGT', 'GGC', 'GGA', 'GGG']),
 ('F', ['TTT', 'TTC']),
 ('I', ['ATT', 'ATC', 'ATA']),
 ('H', ['CAT', 'CAC']),
 ('K', ['AAA', 'AAG']),
 ('M', ['ATG']),
 ('L', ['CTT', 'CTC', 'CTA', 'CTG', 'TTA', ' TTG']),
 ('N', ['AAT', 'AAC']),
 ('Q', ['CAA', 'CAG']),
 ('P', ['CCT', 'CCC', 'CCA', 'CCG']),
 ('S', ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC']),
 ('R', ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG']),
 ('T', ['ACT', 'ACC', 'ACA', 'ACG']),
 ('W', ['TGG']),
 ('V', ['GTT', 'GTC', 'GTA', 'GTG']),
 ('Y', ['TAT', 'TAC']),
 ('Stop', ['TAA', 'TAG', 'TGA'])]


## Feature Development

In [13]:
# ngrams
- 3 grams, which frame?, relationship between subsequent sequence fragments.
... am I next to a fragment

In [1]:
"atg" in "gowijgoijatgsoijdfo"

True

In [21]:
gene_df.sequence[0]

u'TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAATAGTT'

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(1,3), min_df= 1)
d = vectorizer.fit_transform(list(gene_df['sequence']))

In [21]:
def frameShift(seq,shift):
    new_seq = ''
    for j, i in enumerate(seq):
        if j % shift == 0:
            new_seq = new_seq + i + ' '
        else: 
            new_seq = new_seq + i
    return new_seq


fs = [frameShift(seq,3) for seq in gene_df.sequence]

In [47]:
fs[0]

u'T TGT TGA TAT TCT GTT TTT TCT TTT TTA GTT TTC CAC ATG AAA AAT AGT T'

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(3,3), min_df= 1)
d = vectorizer.fit_transform(fs)

In [101]:
fs[0]

u'T TGT TGA TAT TCT GTT TTT TCT TTT TTA GTT TTC CAC ATG AAA AAT AGT T'

In [120]:
x = vectorizer.vocabulary_.copy()

In [121]:
my_dict2 = dict((y,str(x)) for x,y in x.iteritems())

In [122]:
my_dict2

{0: ' a ',
 1: ' aa',
 2: ' ac',
 3: ' ag',
 4: ' at',
 5: ' c ',
 6: ' ca',
 7: ' cc',
 8: ' cg',
 9: ' ct',
 10: ' g ',
 11: ' ga',
 12: ' gc',
 13: ' gg',
 14: ' gt',
 15: ' t ',
 16: ' ta',
 17: ' tc',
 18: ' tg',
 19: ' tt',
 20: 'aa ',
 21: 'aaa',
 22: 'aac',
 23: 'aag',
 24: 'aat',
 25: 'ac ',
 26: 'aca',
 27: 'acc',
 28: 'acg',
 29: 'act',
 30: 'ag ',
 31: 'aga',
 32: 'agc',
 33: 'agg',
 34: 'agt',
 35: 'at ',
 36: 'ata',
 37: 'atc',
 38: 'atg',
 39: 'att',
 40: 'ca ',
 41: 'caa',
 42: 'cac',
 43: 'cag',
 44: 'cat',
 45: 'cc ',
 46: 'cca',
 47: 'ccc',
 48: 'ccg',
 49: 'cct',
 50: 'cg ',
 51: 'cga',
 52: 'cgc',
 53: 'cgg',
 54: 'cgt',
 55: 'ct ',
 56: 'cta',
 57: 'ctc',
 58: 'ctg',
 59: 'ctt',
 60: 'ga ',
 61: 'gaa',
 62: 'gac',
 63: 'gag',
 64: 'gat',
 65: 'gc ',
 66: 'gca',
 67: 'gcc',
 68: 'gcg',
 69: 'gct',
 70: 'gg ',
 71: 'gga',
 72: 'ggc',
 73: 'ggg',
 74: 'ggt',
 75: 'gt ',
 76: 'gta',
 77: 'gtc',
 78: 'gtg',
 79: 'gtt',
 80: 'ta ',
 81: 'taa',
 82: 'tac',
 83: 'tag',
 8

In [56]:
nx = [x.pop(val) for val in vectorizer.vocabulary_ if len(val.strip()) != 3]


In [68]:
gg = list(d)

In [90]:
len(gg)

37049

In [89]:
len(d.indptr)
len(d.data)

1252402

In [76]:
test = gg[0]

In [94]:
import pandas as pd

l = []
for i in range(len(gg)):
    doc = gg[i]
    l.append(pd.DataFrame({"doc":i,"index":doc.indices,"count":doc.data}))
dum = pd.concat(l)
# df2 = pd.DataFrame({"index":doc.indices,"count":doc.data})
# df2


In [99]:
len(set(dum.doc))

37049

In [115]:
dum.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
           dtype='int64', length=1252402)

In [123]:
dum["codon"] = dum['index'].apply(lambda ind: my_dict2[ind])

In [127]:
codonCount = dum[dum.codon.apply(lambda v: len(v.strip())) == 3]
# dum.codon.apply(lambda v: len(v.strip()))

In [128]:
codonCount

,count,doc,index,codon
2,1,0,94,tgt
4,1,0,91,tga
7,1,0,84,tat
10,2,0,89,tct
13,2,0,79,gtt
16,2,0,99,ttt
17,1,0,96,tta
19,1,0,97,ttc
22,1,0,42,cac
25,1,0,38,atg


In [ ]:
ngrams = list(vectorizer.get_feature_names())
[gram.strip() for gram in ngrams if len(gram.strip()) == 3]




In [21]:
df

,a,c,g,t,gene,sequence,start,stop
0,11,5,28,6,False,TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAAT...,True,True
1,14,10,13,13,False,GAAAACAATAGCGGTGTCCCCTTAAAATGGCTTTTCCACAGGTTGT...,True,True
2,19,8,16,7,False,AACCCAAATTAACAGTGTTAATTTATTTTCCACAGGTTGTGGAAAA...,False,True
3,17,6,17,10,False,ACTATTATCCATCGTTCTGTGGAAAACTAGAATAGTTTATGGTAGA...,True,True
4,23,8,11,8,False,TTCTAGAATTATCCACAAGAAGGAACCTAGTATGACTGAAAATGAA...,True,True
5,16,7,15,12,True,TTTTTTGGAACAGGGTCTTGGAATTAGCTCAGAGTCAATTAAAACA...,False,True
6,14,8,19,9,True,ACTTATGAATTTTTTGTTCATGATGCCCGTCTATTAAAGGTCGATA...,True,True
7,20,6,16,8,True,TATTGCAACTATTTACTTAGATCAAATGAAAGAGCTCTTTTGGGAA...,True,True
8,14,7,21,8,True,ATCTTAAAGATGTTATTCTTACTGCTGGTTTTGAAGTTTATAACGC...,True,True
9,17,7,17,9,True,ATTTCTGTTGACTATGTTTTCGAAGAAGACCTAATGATTGAGCAAA...,True,True


In [24]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
codon_to_aa = invert(codon_map)

In [129]:
codon_to_aa

#e

{' TTG': 'L',
 'AAA': 'K',
 'AAC': 'N',
 'AAG': 'K',
 'AAT': 'N',
 'ACA': 'T',
 'ACC': 'T',
 'ACG': 'T',
 'ACT': 'T',
 'AGA': 'R',
 'AGC': 'S',
 'AGG': 'R',
 'AGT': 'S',
 'ATA': 'I',
 'ATC': 'I',
 'ATG': 'M',
 'ATT': 'I',
 'CAA': 'Q',
 'CAC': 'H',
 'CAG': 'Q',
 'CAT': 'H',
 'CCA': 'P',
 'CCC': 'P',
 'CCG': 'P',
 'CCT': 'P',
 'CGA': 'R',
 'CGC': 'R',
 'CGG': 'R',
 'CGT': 'R',
 'CTA': 'L',
 'CTC': 'L',
 'CTG': 'L',
 'CTT': 'L',
 'GAA': 'E',
 'GAC': 'D',
 'GAG': 'E',
 'GAT': 'D',
 'GCA': 'A',
 'GCC': 'A',
 'GCG': 'A',
 'GCT': 'A',
 'GGA': 'G',
 'GGC': 'G',
 'GGG': 'G',
 'GGT': 'G',
 'GTA': 'V',
 'GTC': 'V',
 'GTG': 'V',
 'GTT': 'V',
 'TAA': 'Stop',
 'TAC': 'Y',
 'TAG': 'Stop',
 'TAT': 'Y',
 'TCA': 'S',
 'TCC': 'S',
 'TCG': 'S',
 'TCT': 'S',
 'TGA': 'Stop',
 'TGC': 'C',
 'TGG': 'W',
 'TGT': 'C',
 'TTA': 'L',
 'TTC': 'F',
 'TTT': 'F'}

In [ ]:
def codonToAA(strand):
    aa_seq = ''.join([codon_to_aa(codon) for codon in strand])
    return aa_seq


In [17]:
new_seq

u'T TGT TGA TAT TCT GTT TTT TCT TTT TTA GTT TTC CAC ATG AAA AAT AGT T'

In [19]:
ngrams = list(vectorizer.get_feature_names())
ngrams


[u' ',
 u' a',
 u' aa',
 u' ac',
 u' ag',
 u' at',
 u' c',
 u' ca',
 u' cc',
 u' cg',
 u' ct',
 u' g',
 u' ga',
 u' gc',
 u' gg',
 u' gt',
 u' t',
 u' ta',
 u' tc',
 u' tg',
 u' tt',
 u'a',
 u'a ',
 u'aa',
 u'aa ',
 u'aaa',
 u'aac',
 u'aag',
 u'aat',
 u'ac',
 u'ac ',
 u'aca',
 u'acc',
 u'acg',
 u'act',
 u'ag',
 u'ag ',
 u'aga',
 u'agc',
 u'agg',
 u'agt',
 u'at',
 u'at ',
 u'ata',
 u'atc',
 u'atg',
 u'att',
 u'c',
 u'c ',
 u'ca',
 u'ca ',
 u'caa',
 u'cac',
 u'cag',
 u'cat',
 u'cc',
 u'cc ',
 u'cca',
 u'ccc',
 u'ccg',
 u'cct',
 u'cg',
 u'cg ',
 u'cga',
 u'cgc',
 u'cgg',
 u'cgt',
 u'ct',
 u'ct ',
 u'cta',
 u'ctc',
 u'ctg',
 u'ctt',
 u'g',
 u'g ',
 u'ga',
 u'ga ',
 u'gaa',
 u'gac',
 u'gag',
 u'gat',
 u'gc',
 u'gc ',
 u'gca',
 u'gcc',
 u'gcg',
 u'gct',
 u'gg',
 u'gg ',
 u'gga',
 u'ggc',
 u'ggg',
 u'ggt',
 u'gt',
 u'gt ',
 u'gta',
 u'gtc',
 u'gtg',
 u'gtt',
 u't',
 u't ',
 u'ta',
 u'ta ',
 u'taa',
 u'tac',
 u'tag',
 u'tat',
 u'tc',
 u'tc ',
 u'tca',
 u'tcc',
 u'tcg',
 u'tct',
 u'tg',
 u'tg '

In [ ]:
## amino acid to nucleotide chart http://www.cbs.dtu.dk/courses/27619/codon.html
aa = ["I","L","V","F","M","C","A","G","P","T","S","Y","W","Q","N","H","E","D","K","R","STOP"]
aa_name = ["Isoleucine","Leucine","Valine","Phenylalanine","Methionine","Cysteine","Alanine",
           "Glycine","Proline","Threonine","Serine","Tyrosine","Tryptophan",
           "Glutamine","Asparagine","Histidine","Glutamic_acid", "Aspartic_acid", 
           "Lysine","Arginine","Stop_codons"]
codons = [["ATT", "ATC", "ATA"],["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          ["GTT", "GTC", "GTA", "GTG"],["TTT", "TTC"],["ATG"],["TGT", "TGC"],
          ["GCT", "GCC", "GCA", "GCG"],["GGT", "GGC", "GGA", "GGG"],
          ["CCT", "CCC", "CCA", "CCG"],
          ["ACT", "ACC", "ACA", "ACG"],["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          ["TAT", "TAC"],["TGG"],["CAA", "CAG"],["AAT", "AAC"],["CAT", "CAC"],["GAA", "GAG"],
          ["GAT", "GAC"],["AAA", "AAG"],["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],
          ["TAA", "TAG", "TGA"]]

codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q"["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
# aa_properties
Aliphatic: Alanine,Isoleucine,Leucine,Valine
Aromatic: Phenylalanine, Tryptophan, Tyrosine
Polar_neutral: Asparagine, Cysteine, Glutamine, Methionine,Serine,Threonine
charged: Aspartic acid, Glutamic acid
    
Properties table
https://www.thermofisher.com/us/en/home/life-science/protein-biology/protein-biology-learning-center/protein-biology-resource-library/pierce-protein-methods/amino-acid-physical-properties.html

http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3456822/pdf/10867_2004_Article_396406.pdf

http://www.proteinstructures.com/Structure/Structure/amino-acids.html

In [10]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
codon_to_aa = invert(codon_map)

In [11]:
codon_to_aa

{' TTG': 'L',
 'AAA': 'K',
 'AAC': 'N',
 'AAG': 'K',
 'AAT': 'N',
 'ACA': 'T',
 'ACC': 'T',
 'ACG': 'T',
 'ACT': 'T',
 'AGA': 'R',
 'AGC': 'S',
 'AGG': 'R',
 'AGT': 'S',
 'ATA': 'I',
 'ATC': 'I',
 'ATG': 'M',
 'ATT': 'I',
 'CAA': 'Q',
 'CAC': 'H',
 'CAG': 'Q',
 'CAT': 'H',
 'CCA': 'P',
 'CCC': 'P',
 'CCG': 'P',
 'CCT': 'P',
 'CGA': 'R',
 'CGC': 'R',
 'CGG': 'R',
 'CGT': 'R',
 'CTA': 'L',
 'CTC': 'L',
 'CTG': 'L',
 'CTT': 'L',
 'GAA': 'E',
 'GAC': 'D',
 'GAG': 'E',
 'GAT': 'D',
 'GCA': 'A',
 'GCC': 'A',
 'GCG': 'A',
 'GCT': 'A',
 'GGA': 'G',
 'GGC': 'G',
 'GGG': 'G',
 'GGT': 'G',
 'GTA': 'V',
 'GTC': 'V',
 'GTG': 'V',
 'GTT': 'V',
 'TAA': 'Stop',
 'TAC': 'Y',
 'TAG': 'Stop',
 'TAT': 'Y',
 'TCA': 'S',
 'TCC': 'S',
 'TCG': 'S',
 'TCT': 'S',
 'TGA': 'Stop',
 'TGC': 'C',
 'TGG': 'W',
 'TGT': 'C',
 'TTA': 'L',
 'TTC': 'F',
 'TTT': 'F'}

## Model testing

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, classification_report


from sklearn.learning_curve import learning_curve

from sklearn.cross_validation import train_test_split, KFold, cross_val_score

In [86]:
# x = df[["a","c","g","t","start","stop"]]
y = df.gene
x = aa_counts.drop('gene', 1)

In [87]:
algorithms = [RandomForestClassifier(), DecisionTreeClassifier(), GaussianNB(), 
              SVC(), KNeighborsClassifier(n_neighbors=6), LogisticRegression()]

for algo in algorithms:
    accuracy = cross_val_score(algo,x,y)
    print("{:s} Accuracy Score : {:f}".format(str(algo).split('(', 1)[0],accuracy.mean()))

RandomForestClassifier Accuracy Score : 0.675565
DecisionTreeClassifier Accuracy Score : 0.613944
GaussianNB Accuracy Score : 0.673784
SVC Accuracy Score : 0.734946
KNeighborsClassifier Accuracy Score : 0.667900
LogisticRegression Accuracy Score : 0.733029
